# Import Essentials:

In [7]:
from PUGNN import SW, HomoDataset, HomoDataReader, Trainer, DataLoader
import numpy as np

# Specify the `output` and `input` directories:

In [8]:
input_directory  = "/eos/user/m/mjalalva/Run1/jun4/"
output_directory = './'

# Define a model and some metrics:

In [9]:
from torch_geometric.nn import GraphConv, MLP, global_add_pool, GATv2Conv, LayerNorm, global_mean_pool
import torch
import torch.nn.functional as F


class PUModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_features, GNN=GraphConv):
        super(PUModel, self).__init__()
        torch.manual_seed(12345)
        self.num_features = num_features
        
        self.conv1 = GNN(in_channels, hidden_channels, edge_dim=1, add_self_loops=False)
        self.norm1 = LayerNorm(hidden_channels)
        self.mlp1  = MLP([hidden_channels + num_features, 2*hidden_channels, 2*hidden_channels], norm='layer_norm')
        
        self.conv2 = GNN(hidden_channels, hidden_channels, edge_dim=1, add_self_loops=False)
        self.norm2 = LayerNorm(hidden_channels)
        self.mlp2  = MLP([3*hidden_channels + num_features, 2*hidden_channels, hidden_channels, hidden_channels//2, out_channels], norm='layer_norm')
        
    def forward(self, data):
        x, adj, features, batch = data.x, data.adj_t, torch.reshape(data.features, (-1, self.num_features)), data.batch
        
        # 1. Obtain node embeddings 
        x = self.conv1(x, adj)
        x = self.norm1(x)
        x = x.relu()
        
        g = self.mlp1(torch.cat([global_mean_pool(x, batch), features], dim=1))
    
        x = self.conv2(x, adj)
        x = self.norm2(x)
        x = x.relu()

        g = self.mlp2(torch.cat([global_mean_pool(x, batch), g, features], dim=1))
        
        return g
    
class Bias(torch.nn.Module):
    def __init__(self):
        super().__init__()
        return
    
    def forward(self, input, target):
        return (input-target).mean()
    
class MAPE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        return
        
    def forward(self, input, target):
        return (abs(input-target)/target).mean()

In [10]:
model = PUModel(in_channels=15, hidden_channels=32, num_features=7, out_channels=1, GNN=GATv2Conv)

# Make an Instance of `SW` Class with Your `DataSet` and a `DataLoader` Class:

In [ ]:
software1 = SW(input_directory, output_directory, name='pu-test')
sample_metadata = dict(zip(
    map(str, list(range(20, 81))), # Range on PU values
    np.ones(61) * 100               # Number of events at a given PU
))

software1.set_dataset(HomoDataset,      # Your Dataset Type
                      sample_metadata, 
                      HomoDataReader()  # A Data Reader Function
                     )

software1.set_loader(DataLoader, # Your Dataloader Type
#                      loading_workers=4,
                     batch_size=4,
                     num_workers=16
                    )

Preprocessing...
1 found at '/eos/user/m/mjalalva/Run1/jun4/'
PU = 020 overflows... Setting PU = 020 to 67 instead of 100.0
PU = 021 overflows... Setting PU = 021 to 89 instead of 100.0
PU = 078 overflows... Setting PU = 078 to 82 instead of 100.0
PU = 079 overflows... Setting PU = 079 to 59 instead of 100.0
PU = 080 overflows... Setting PU = 080 to 69 instead of 100.0
Initializing...


loading ...:   0%|          | 0/1491 [00:00<?, ?Batch/s]

preparing directory 'test' ...:   0%|          | 0/149 [00:00<?, ?Batch/s]

/cvmfs/sft.cern.ch/lcg/views/LCG_104a_cuda/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/cvmfs/sft.cern.ch/lcg/views/LCG_104a_cuda/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
/cvmfs/sft.cern.ch/lcg/views/LCG_104a_cuda/x86_64-centos7-gcc11-opt/lib/

# Enter into the `TrainerScope` of Your `SW`:

In [ ]:
with software1.trainer_scope(Trainer) as pu_trainer:
    pu_trainer.set(model)
    res = pu_trainer.train(
        max_epochs=5, optimizer=torch.optim.RAdam,
        optimizer_args=dict(lr=5e-3),
        loss_fn=torch.nn.L1Loss,
        metrics=[MAPE(), Bias()], select_topk=5,
        lr_scheduler=torch.optim.lr_scheduler.MultiStepLR,
        lr_scheduler_args=dict(milestones=[7, 15, 25, 35], gamma=0.06),
    )

# After Training, Let's Enter into the `AnalyzerScope`:

In [10]:
eval_model = PUModel(in_channels=15, hidden_channels=150, num_features=7, out_channels=1, GNN=GATv2Conv)

with software1.analyzer_scope() as pu_analyzer:
    pu_analyzer(eval_model, res.models, torch.nn.L1Loss())
    
    best_model = pu_analyzer.model
    res_plot   = pu_analyzer.residual_plot()
    metrics    = pu_analyzer.apply_metrics([Bias(), MAPE()])
    dist_plots = pu_analyzer.distribution_plots()
    LLERes     = pu_analyzer.rangeLLE(30,60)
    
    no_verts   = pu_analyzer.extract_feature(0, 7)
    comparing  = pu_analyzer.compare(
        # Here, you can add the outputs as follows:
        # `model2` = (model2_summary.y, model2_summary.yhat)
        # `model3` = ...
        NV = (pu_analyzer.y, no_verts)
    )

processing...


Evaluating...:   0%|          | 0/16 [00:00<?, ?Batch/s]

Range-Based LLE:   0%|          | 0/31 [00:00<?, ?PU/s]

PU = 30 Indexing...:   0%|          | 0/61 [00:00<?, ?Graph/s]

Change factor variable `fraction` (current value is `0.6`) based on the following:
	Poisson distribution(30) for PU = 27.0 is violated by 1 count less than expected
	Poisson distribution(30) for PU = 28.0 is violated by 1 count less than expected
	Poisson distribution(30) for PU = 32.0 is violated by 1 count less than expected
	Poisson distribution(30) for PU = 36.0 is violated by 1 count less than expected
/eos/home-i04/c/cmstandi/SWAN_projects/saleh-highlights/PUGNN/PUGNN/utils/postprocessing_tools.py:8: RuntimeWarning:

invalid value encountered in log



RuntimeError: Error interval Computation Failed

## Distribution plots

In [ ]:
dist_plots.heatmap

In [ ]:
dist_plots.histogram

In [ ]:
dist_plots.kdeplot

## Residual plot

In [ ]:
res_plot

## Compare the model with another models

In [ ]:
comparing.plot   # Visualize the comparasion

In [ ]:
comparing.R2     # R2 factor of model itself and with other models if they exist

## Log-likelihood Estimation through a Given Range of $<PU>=L\sigma$

In [ ]:
LLERes.plot  # The plot of the result

In [ ]:
LLERes.estimated_pu       # Estimated <PU> by model

In [ ]:
LLERes.true_pu            # Expected or real <PU>

In [ ]:
LLERes.lower_bond_error   # Lower-bond error of estimation

In [ ]:
LLERes.upper_bond_error   # Upper-bond error of estimation